In [1]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from dataset import *
from vit import *
import torch.optim as optim
from tqdm import tqdm
from train import *
from torch.utils.data import DataLoader


c:\Users\Jun Khai\miniconda3\envs\DeepLearning\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [2]:

train_imgs, test_imgs = load_adni_images(verbose = False)

train_set = ADNIDataset(train_imgs[0], transform=standardTransform)
val_set = ADNIDataset(train_imgs[1], transform=standardTransform)
test_set = ADNIDataset(test_imgs[0], transform=standardTransform)

train_loader = DataLoader(train_set, batch_size=2)
val_loader = DataLoader(val_set, batch_size=2)
test_loader = DataLoader(test_set, batch_size=2)

In [4]:
def train(model, train_loader, val_loader, criterion, n_epochs, lr):
  # Define and optimizer
  optimizer = optim.Adam(model.parameters(), lr=lr)

  # Train the model
  for epoch in range(n_epochs):
    # Train the model for one epoch
    model.train()
    train_loss = 0.0
    for imgs, labels in tqdm(train_loader):
      optimizer.zero_grad()
      out = model(imgs)
      loss = criterion(out, labels)
      loss.backward()
      optimizer.step()
      train_loss += loss.item() * imgs.size(0)
    train_loss /= len(train_loader.dataset)

    # Evaluate the model on the validation set
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
      for imgs, labels in tqdm(val_loader):
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * imgs.size(0)
        _, preds = torch.max(outputs, 1)
        val_acc += torch.sum(preds == labels.data)
    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader.dataset)

    # Print the loss and accuracy for this epoch
    print('Epoch [{}/{}], Train Loss: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}'
        .format(epoch+1, n_epochs, train_loss, val_loss, val_acc))
  
model = ViT(image_size=128, patch_size=32, num_classes=2, dim=1024, depth=6, heads=16, mlp_dim=2048, dropout=0.1)
print(model)
train(model, train_loader, val_loader, nn.CrossEntropyLoss(), n_epochs=10, lr=0.001)

ViT(
  (patch_to_embedding): Linear(in_features=3072, out_features=1024, bias=True)
  (transformer): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
      )
      (linear1): Linear(in_features=1024, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=1024, bias=True)
      (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (to_cls_token): Identity()
  (mlp_head): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=2048, out_features=2, bias=True)
  )
)


  0%|          | 0/8610 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 must have the same dtype